<a href="https://colab.research.google.com/github/alfcastillo90/pyspark_streaming_ubo/blob/main/pyspark_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## descargamos e instalamos jdk y apache spark
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget 'https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz' -O 'spark-3.3.1-bin-hadoop3.tgz'
!tar -xvf /content/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark 
import os 
import findspark 
## creamos las variables de entorno JAVA_HOME y SPARK_HOME
os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.3.1-bin-hadoop3'

In [ ]:
## inicializamos
findspark.init()

In [ ]:
## nos integramos con google drive
from google.colab import drive 
drive.mount("/content/gdrive")
gdrive='content/gdrive/MyDrive/Estudios/UBO/BIGDATA'

In [ ]:
## creamos la carpeta streaming, primero validamos si existe o no
!test -d /content/streaming && echo 'la carpeta existe' || mkdir streaming

In [ ]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.master('local').appName('PySparkStreaming').config('spark.ui.port',4050).getOrCreate()

In [ ]:
## Definimos el esquema

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

structureSchema = StructType([ \
                     StructField('index', IntegerType(), True), \
                     StructField('Order ID', StringType(), True), \
                     StructField('Date', StringType(), True), \
                     StructField('Status', StringType(), True), \
                     ])


In [ ]:
## Leemos el archivo csv y le asignamos el esquema que fue definido previamente
dataFrame = spark.read.csv('/'+gdrive+'/amazon-orders-status.csv', sep=',', header=True, schema=structureSchema)


In [ ]:
dataFrame.printSchema()

In [ ]:
streamingDataFrame = spark.readStream.format('csv').schema(structureSchema).option('header', True).load('streaming')

In [ ]:
streamingDataFrame.isStreaming